In [1]:
import os

In [11]:
hello_world_script_file=os.path.join(os.path.pardir,'src','models','hello_world_api.py')

In [13]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return "hello {0}".format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=False)

Overwriting ..\src\models\hello_world_api.py


In [14]:
import json
import requests

In [16]:
url='http://127.0.0.1:10001/api'
data=json.dumps({'name':'Diwakar'})
r=requests.post(url,data)

In [17]:
print(r.text)

hello Diwakar


# Machine learning API

In [22]:
machine_learning_script_file=os.path.join(os.path.pardir,'src','models','machine_learning_script.py')

In [28]:
%%writefile $machine_learning_script_file
from flask import Flask,request
import pandas as pd
import numpy as np
import json
import pickle
import os

app=Flask(__name__)
model_path=os.path.join(os.path.pardir,os.path.pardir,'models')
model_file=os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file=os.path.join(os.path.pardir,'models','lr_scaler.pkl')

scaler=pickle.load(open(scaler_file))
model=pickle.load(open(model_file))

columns = [ u'Age', u'Fare', u'FamilySize', \
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 
@app.route('/api', methods=['POST'])
def makepredictions():
    data=json.dumps(requests.get_json(force=True))
    df=pd.read_json(data)
    passenger_ids=df['PassengerId'].ravel()
    actuals=df['Survived'].ravel()
    X=df[columns].asmatrix().astype('float')
    X_scaled=scaler.transfrom(X)
    predictions=model.predict(X_scaled)
    df_response=pd.DataFrame({'PassengerId':passenger_ids, 'Predicted':predictions,'Actual':actuals})
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=10001, debug=False)                 
                   

Overwriting ..\models\machine_learning_script.py


## Invoking API using Requests

In [31]:
import os
import pandas as pd
processed_data_path=os.path.join(os.path.pardir,'data','processed')
train_file_path=os.path.join(processed_data_path,'train.csv')
train_df=pd.read_csv(train_file_path)
test_df=os.path.join(processed_data_path,'test.csv')

In [34]:
survived_passengers=train_df[train_df['Survived'] == 1][:5]

In [35]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [41]:
import requests
def make_api_request(data):
    url='http://127.0.0.1:10001/api'
    r=requests.post(url,data)
    return(r.json())

In [42]:
make_api_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [43]:
result=make_api_request(train_df.to_json())
df_result=pd.read_json(json.dumps(result))
df_result.head()

,PassengerId,Predicted,Actual
0,1,0,0
1,2,1,1
10,108,0,1
100,190,0,0
101,20,1,1


In [44]:
import numpy as np

In [47]:
np.mean(df_result.Actual == df_result.Predicted)

0.8383838383838383